# Preamble
Setting up:
 - Required modules
 - Folder paths
 - Useful functions

In [1]:
# Required imports

import os
import json
import numpy as np
import pandas as pd
import random
import imageio
import magnum as mn
from PIL import Image
from matplotlib import pyplot as plt

import habitat_sim
from habitat_sim.utils import common as utils
from habitat_sim.utils import viz_utils as vut

In [2]:
# setting up folder paths
habitat_dir_path = "/home/wes/habitat/data/"
scene_dir_path = os.path.join(habitat_dir_path, f"scene_datasets/mp3d/")
rxr_data_path = "/media/wes/Lexar/rxr_data/rxr-data/"
output_path = "/home/wes/Documents/git_ws/OCOM5300M_ResearchProject/outputs/"

annotations_fname = "rxr_train_guide.jsonl/rxr_train_guide.json"
annotations_path = os.path.join(rxr_data_path, annotations_fname)

mp3d_scene_config = os.path.join(scene_dir_path, f"mp3d.scene_dataset_config.json")

def get_scene_path(scene_dir_path, scene_id):
    return os.path.join(scene_dir_path, f"{scene_id}/{scene_id}.glb")

In [3]:
scene_id = "17DRP5sb8fy"
scene_path = get_scene_path(scene_dir_path, scene_id)

print(f"mp3d_scene_config: {mp3d_scene_config}, exists: {os.path.exists(mp3d_scene_config)}")
print(f"scene_path: {scene_path}, exists: {os.path.exists(scene_path)}")
print(f"annotations_path: {annotations_path}, exists: {os.path.exists(annotations_path)}")

mp3d_scene_config: /home/wes/habitat/data/scene_datasets/mp3d/mp3d.scene_dataset_config.json, exists: True
scene_path: /home/wes/habitat/data/scene_datasets/mp3d/17DRP5sb8fy/17DRP5sb8fy.glb, exists: True
annotations_path: /media/wes/Lexar/rxr_data/rxr-data/rxr_train_guide.jsonl/rxr_train_guide.json, exists: True


## Preamble: Utility Functions
### Display observations

In [4]:
# Change to do something like this maybe: https://stackoverflow.com/a/41432704
#    i.e. show multiple subplots on one plot
def display_sample(rgb_obs, semantic_obs=np.array([]), depth_obs=np.array([])):
    from habitat_sim.utils.common import d3_40_colors_rgb

    rgb_img = Image.fromarray(rgb_obs, mode="RGBA")

    arr = [rgb_img]
    titles = ["rgb"]
    if semantic_obs.size != 0:
        semantic_img = Image.new("P", (semantic_obs.shape[1], semantic_obs.shape[0]))
        semantic_img.putpalette(d3_40_colors_rgb.flatten())
        semantic_img.putdata((semantic_obs.flatten() % 40).astype(np.uint8))
        semantic_img = semantic_img.convert("RGBA")
        arr.append(semantic_img)
        titles.append("semantic")

    if depth_obs.size != 0:
        depth_img = Image.fromarray((depth_obs / 10 * 255).astype(np.uint8), mode="L")
        arr.append(depth_img)
        titles.append("depth")

    plt.figure(figsize=(12, 8))
    for i, data in enumerate(arr):
        ax = plt.subplot(1, 3, i + 1)
        ax.axis("off")
        ax.set_title(titles[i])
        plt.imshow(data)
    plt.show(block=False)


if __name__ == "__main__":
    import argparse

    parser = argparse.ArgumentParser()
    parser.add_argument("--no-display", dest="display", action="store_false")
    parser.add_argument("--no-make-video", dest="make_video", action="store_false")
    parser.set_defaults(show_video=True, make_video=True)
    args, _ = parser.parse_known_args()
    show_video = args.display
    display = args.display
    do_make_video = args.make_video
else:
    show_video = False
    do_make_video = False
    display = False

# import the maps module alone for topdown mapping
if display:
    from habitat.utils.visualizations import maps

PluginManager::Manager: duplicate static plugin StbImageImporter, ignoring
PluginManager::Manager: duplicate static plugin GltfImporter, ignoring
PluginManager::Manager: duplicate static plugin BasisImporter, ignoring
PluginManager::Manager: duplicate static plugin AssimpImporter, ignoring
PluginManager::Manager: duplicate static plugin AnySceneImporter, ignoring
PluginManager::Manager: duplicate static plugin AnyImageImporter, ignoring


### Configuring Simulator Settings

In [7]:
# Default sim settings
rgb_sensor = True
depth_sensor = True
semantic_sensor = False
seed = 1

sim_settings = {
    "width": 256,  # Spatial resolution of the observations
    "height": 256,
    "scene": scene_path,  # Scene path
    "scene_dataset": mp3d_scene_config,  # the scene dataset configuration files
    "default_agent": 0,
    "sensor_height": 1.5,  # Height of sensors in meters
    "color_sensor": rgb_sensor,  # RGB sensor
    "depth_sensor": depth_sensor,  # Depth sensor
    "semantic_sensor": semantic_sensor,  # Semantic sensor
    "seed": seed,  # used in the random navigation
    "enable_physics": False,  # kinematics only    
}

In [8]:
# Sim config factory?
def make_cfg(settings):
    sim_cfg = habitat_sim.SimulatorConfiguration()
    sim_cfg.gpu_device_id = 0
    sim_cfg.scene_id = settings["scene"]
    sim_cfg.scene_dataset_config_file = settings["scene_datasets"]
    sim_cfg.enable_physics = settings["enable_physics"]

    # sensor specs
    sensor_specs = []

    color_sensor_spec = habitat_sim.CameraSensorSpec()
    color_sensor_spec.uuid = "color_sensor"
    color_sensor_spec.sensor_type = habitat_sim.SensorType.COLOR
    color_sensor_spec.resolution = [settings["height"], settings["width"]]
    color_sensor_spec.position = [0.0, settings["sensor_height"], 0.0]
    color_sensor_spec.sensor_subtype = habitat_sim.SensorSubType.FISHEYE
    sensor_specs.append(color_sensor_spec)

    depth_sensor_spec = habitat_sim.CameraSensorSpec()
    depth_sensor_spec.uuid = "depth_sensor"
    depth_sensor_spec.sensor_type = habitat_sim.SensorType.DEPTH
    depth_sensor_spec.resolution = [settings["height"], settings["width"]]
    depth_sensor_spec.position = [0.0, settings["sensor_height"], 0.0]
    depth_sensor_spec.sensor_subtype = habitat_sim.SensorSubType.FISHEYE
    sensor_specs.append(depth_sensor_spec)

    #agent movement
    agent_cfg = habitat_sim.agent.AgentConfiguration()
    agent_cfg.sensor_specifications = sensor_specs
    agent_cfg.action_space = {
        "move_forward" : habitat_sim.agent.ActionSpec(
            "move_forward", habitat_sim.agent.ActuationSpec(amount=0.25)
        ),
        "turn_left":habitat_sim.agent.ActionSpec(
            "turn_left", habitat_sim.agent.ActuationSpec(amount=30.0)
        ),
        "turn_right":habitat_sim.agent.ActionSpec(
            "turn_right", habitat_sim.agent.ActuationSpec(amount=30.0)
        )
    }
    return habitat_sim.Configuration(sim_cfg, [agent_cfg])